# Aprendizaje

In [1]:
import numpy as np
import torch

* Crear un algoritmo que _aprende_ de la data
* Hacer un _fit_ a la data
* El proceso: Una funcion con un numero de parametros desconocidos cuyos valores son estimados de la data
* Un modelo

Pytorch esta diseniado para faciliatar la creacion de modelos para los cuales las derivadas del error con respecto a los parametros pueden ser expresadas de forma analitica.

## El aprendizaje es la estimacion de parametros

* Data
* Escoger un modelo
* Estimar los parametros del modelo para tener buenas predicciones sobre data nueva.

## El problema

Tenemos un termometro que no ensenia las unidades en las cuales muestra la temperatura.

* data: lecturas del termometro y los valores correspondientes en una unidad conocida.
* Escoger un modelo
* Ajustar los parametros iterativamente hasta ue la medida del error sea lo suficientemente baja

In [2]:
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0] # Temperatura en grados celsios
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4] # Unidades desconocidas
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

### Escogiendo nuestro primer modelo

t_c = w * t_u + b

**Spoiler**: Sabemos que un modelo linear es el modelo correcto.

* Tenemos un modelo con parametros desconocidos: $w$ y $b$
* Tenemos que estimar esos parametros para que el error entre $\hat{y}$ y $y$ sea lo mas pequenio posible
* Todavia no hemos definido una medida de ese error.
    * Esta medida, _loss function_, deberia ser alta si el error es alto e idealmente deberia ser lo mas bajo posible cuando haya un match perfecto.
* Nuestro proceso de optimizacion deberia encontrar $w$ y $b$ para que el _loss function_ este en un minimo.

### Loss function

* Una funcion que calcula un valor numerico que el proceso de optimizacion va a intentar minimizar.
* El calculo del _loss_ normalmente involucra tomar la diferencia entre el output deseado para alguna muestra de entrenamiento y el verdadero output producido por el modelo cuando ve esos outputs.
* En nuestro caso: `t_p - t_c`
* Nuestra _loss_function_ deberia siempre ser positiva
* Conceptualmente un _loss function_ es una forma de priorizar cuales errores de nuestro training sample arreglar, para que los ajustes a los parametros resulten en ajustes a los outputs para las muestras con mayor peso.


Ejemplos:
* $|t_p - t_c|$
* $(t_p - t_c)^2$

## Problema a PyTorch

In [3]:
def model(t_u, w, b):
    return w * t_u + b


def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [4]:
w = torch.ones(1)
b = torch.zeros(1)

t_p = model(t_u, w, b)
t_p

tensor([35.7000, 55.9000, 58.2000, 81.9000, 56.3000, 48.9000, 33.9000, 21.8000,
        48.4000, 60.4000, 68.4000])

In [5]:
loss = loss_fn(t_p, t_c)
loss

tensor(1763.8846)

## La gradiente

**Como estimamos $w$ y $b$ para que el _loss_ llegue al minimo?**

* _Gradient descent_
    * Calcular la razon de cambio del _loss_ con respecto a cada parametro
    * Aplicar un cambio a cada parametro en la direccion que reduzca el _loss_

In [6]:
delta = 0.1

loss_rate_of_change_w = (loss_fn(model(t_u, w + delta, b), t_c) -
                        loss_fn(model(t_u, w - delta, b), t_c)) / (2.0 * delta)

**Conceptualmente:** 
* Un incremento unitario en $w$ va a provocar un cambio en el _loss_. 
    * Si el cambio es negativo, tenemos que incrementar $w$ para minimizar el _loss_
    * Si el cambio es positivo, tenemos que reducir $w$
    
**Por cuanto debemos de incrementar o reducir el valor de $w$?**
* Proporcional a la razon de cambio del _loss_

In [7]:
learning_rate = 1e-2

w = w - learning_rate * loss_rate_of_change_w

In [8]:
loss_rate_of_change_b = (loss_fn(model(t_u, w, b + delta), t_c) -
                        loss_fn(model(t_u, w, b - delta), t_c) / (2.0 * delta))

b = b - learning_rate * loss_rate_of_change_b

**Esto representa un paso en gradient descent**

## Version analitica

* Que pasa si el valor de delta fuera infinitesimamente pequenio?
* La derivada del _loss_ con respecto a cada parametro.
    * $\frac{\partial L}{\partial w} = \frac{\partial L}{\partial t_p} \frac{\partial t_p}{\partial w}$

In [9]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c) ** 2
    return squared_diffs.mean()

$\frac{\partial x^2}{\partial x} = 2x$

In [10]:
def dloss_fn(t_p, t_c):
    dsq_diffs = 2 * (t_p - t_c)
    return dsq_diffs

In [11]:
def model(t_u, w, b):
    return w * t_u + b


def dmodel_dw(t_u, w, b):
    return t_u


def dmodel_db(t_u, w, b):
    return 1.0

La funcion retornando la gradiente del _loss_ con respecto a $w$ y $b$

In [12]:
def grad_fn(t_u, t_c, t_p, w, b):
    dloss_dw = dloss_fn(t_p, t_c) * dmodel_dw(t_u, w, b)
    dloss_db = dloss_fn(t_p, t_c) * dmodel_db(t_u, w, b)
    return torch.stack([dloss_dw.mean(), dloss_db.mean()])

## Training loop

Ya tenemos todo para optimizar nuestros parametros:
* Empezamos con un valor tentativo para cada paremetro
* Actualizamos iterativamente
* Paramos despues de un numero fijo de iteraciones
* o hasta que $w$ y $b$ dejen de cambiar


**Epoch**: una iteracion de entrenamiento durante la cual actualizamos los parametros para todo nuestro dataset de entrenamiento

In [13]:
def training_loop(model, n_epochs, learning_rate, params, t_u, t_c, print_params=True):
    for epoch in range(1, n_epochs + 1):
        w, b = params
        
        t_p = model(t_u, w, b) # Forward pass
        loss = loss_fn(t_p, t_c)
        grad = grad_fn(t_u, t_c, t_p, w, b) # Backward pass
        
        params = params - learning_rate * grad
        
        if epoch % 10 == 0 or epoch == 1:
            print(f"Epoch {epoch}, Loss {loss}")
            if print_params:
                print(f"\tParams: {params}")
                print(f"\tGrad: {grad}")
        
    return params

In [14]:
training_loop(model,
              n_epochs = 100,
              learning_rate= 1e-2,
              params = torch.tensor([1.0, 0.0]),
              t_u = t_u,
              t_c = t_c)

Epoch 1, Loss 1763.8846435546875
	Params: tensor([-44.1730,  -0.8260])
	Grad: tensor([4517.2964,   82.6000])
Epoch 10, Loss 9.090107547845813e+34
	Params: tensor([3.2144e+17, 5.6621e+15])
	Grad: tensor([-3.2700e+19, -5.7600e+17])
Epoch 20, Loss inf
	Params: tensor([1.3457e+35, 2.3704e+33])
	Grad: tensor([-1.3690e+37, -2.4114e+35])
Epoch 30, Loss nan
	Params: tensor([nan, nan])
	Grad: tensor([nan, nan])
Epoch 40, Loss nan
	Params: tensor([nan, nan])
	Grad: tensor([nan, nan])
Epoch 50, Loss nan
	Params: tensor([nan, nan])
	Grad: tensor([nan, nan])
Epoch 60, Loss nan
	Params: tensor([nan, nan])
	Grad: tensor([nan, nan])
Epoch 70, Loss nan
	Params: tensor([nan, nan])
	Grad: tensor([nan, nan])
Epoch 80, Loss nan
	Params: tensor([nan, nan])
	Grad: tensor([nan, nan])
Epoch 90, Loss nan
	Params: tensor([nan, nan])
	Grad: tensor([nan, nan])
Epoch 100, Loss nan
	Params: tensor([nan, nan])
	Grad: tensor([nan, nan])


tensor([nan, nan])

### Que paso?

* Como podemos limitar la magnitud de `learning_rate * grad`?

In [15]:
training_loop(model,
              n_epochs = 100,
              learning_rate= 1e-4,
              params = torch.tensor([1.0, 0.0]),
              t_u = t_u,
              t_c = t_c)

Epoch 1, Loss 1763.8846435546875
	Params: tensor([ 0.5483, -0.0083])
	Grad: tensor([4517.2964,   82.6000])
Epoch 10, Loss 29.105241775512695
	Params: tensor([ 0.2324, -0.0166])
	Grad: tensor([1.4803, 3.0544])
Epoch 20, Loss 29.095884323120117
	Params: tensor([ 0.2323, -0.0196])
	Grad: tensor([-0.0531,  3.0268])
Epoch 30, Loss 29.08671760559082
	Params: tensor([ 0.2323, -0.0226])
	Grad: tensor([-0.0533,  3.0263])
Epoch 40, Loss 29.077558517456055
	Params: tensor([ 0.2324, -0.0256])
	Grad: tensor([-0.0533,  3.0258])
Epoch 50, Loss 29.068401336669922
	Params: tensor([ 0.2325, -0.0287])
	Grad: tensor([-0.0532,  3.0252])
Epoch 60, Loss 29.05925178527832
	Params: tensor([ 0.2325, -0.0317])
	Grad: tensor([-0.0533,  3.0247])
Epoch 70, Loss 29.050098419189453
	Params: tensor([ 0.2326, -0.0347])
	Grad: tensor([-0.0532,  3.0242])
Epoch 80, Loss 29.04095458984375
	Params: tensor([ 0.2326, -0.0377])
	Grad: tensor([-0.0532,  3.0236])
Epoch 90, Loss 29.03180503845215
	Params: tensor([ 0.2327, -0.0408

tensor([ 0.2327, -0.0438])

### Mejor

El comportamiento se mantuvo estable

Hay otro problema:
* la gradiente

```
Epoch 1, Loss 1763.8846435546875
	Params: tensor([ 0.5483, -0.0083])
	Grad: tensor([4517.2964,   82.6000])
```

In [16]:
4517.2964/82.6000

54.68881840193705

In [17]:
t_un = 0.1 * t_u

In [18]:
training_loop(model,
              n_epochs = 100,
              learning_rate= 1e-2,
              params = torch.tensor([1.0, 0.0]),
              t_u = t_un, # normalizado
              t_c = t_c)

Epoch 1, Loss 80.36434173583984
	Params: tensor([1.7761, 0.1064])
	Grad: tensor([-77.6140, -10.6400])
Epoch 10, Loss 29.030487060546875
	Params: tensor([ 2.3232, -0.0710])
	Grad: tensor([-0.5355,  2.9295])
Epoch 20, Loss 28.157800674438477
	Params: tensor([ 2.3746, -0.3615])
	Grad: tensor([-0.5093,  2.8832])
Epoch 30, Loss 27.314294815063477
	Params: tensor([ 2.4251, -0.6471])
	Grad: tensor([-0.5007,  2.8346])
Epoch 40, Loss 26.498987197875977
	Params: tensor([ 2.4747, -0.9280])
	Grad: tensor([-0.4923,  2.7868])
Epoch 50, Loss 25.710935592651367
	Params: tensor([ 2.5235, -1.2040])
	Grad: tensor([-0.4840,  2.7398])
Epoch 60, Loss 24.949237823486328
	Params: tensor([ 2.5714, -1.4755])
	Grad: tensor([-0.4758,  2.6936])
Epoch 70, Loss 24.212995529174805
	Params: tensor([ 2.6186, -1.7423])
	Grad: tensor([-0.4678,  2.6482])
Epoch 80, Loss 23.501379013061523
	Params: tensor([ 2.6649, -2.0047])
	Grad: tensor([-0.4599,  2.6035])
Epoch 90, Loss 22.813547134399414
	Params: tensor([ 2.7105, -2.262

tensor([ 2.7553, -2.5162])

In [19]:
params = training_loop(model,
                       n_epochs = 5000,
                       learning_rate= 1e-2,
                       params = torch.tensor([1.0, 0.0]),
                       t_u = t_un,
                       t_c = t_c,
                       print_params=False)

params

Epoch 1, Loss 80.36434173583984
Epoch 10, Loss 29.030487060546875
Epoch 20, Loss 28.157800674438477
Epoch 30, Loss 27.314294815063477
Epoch 40, Loss 26.498987197875977
Epoch 50, Loss 25.710935592651367
Epoch 60, Loss 24.949237823486328
Epoch 70, Loss 24.212995529174805
Epoch 80, Loss 23.501379013061523
Epoch 90, Loss 22.813547134399414
Epoch 100, Loss 22.148710250854492
Epoch 110, Loss 21.506103515625
Epoch 120, Loss 20.88498306274414
Epoch 130, Loss 20.284622192382812
Epoch 140, Loss 19.70433807373047
Epoch 150, Loss 19.14344596862793
Epoch 160, Loss 18.6013126373291
Epoch 170, Loss 18.077299118041992
Epoch 180, Loss 17.57081413269043
Epoch 190, Loss 17.081254959106445
Epoch 200, Loss 16.608064651489258
Epoch 210, Loss 16.150693893432617
Epoch 220, Loss 15.708612442016602
Epoch 230, Loss 15.281317710876465
Epoch 240, Loss 14.868304252624512
Epoch 250, Loss 14.469095230102539
Epoch 260, Loss 14.083236694335938
Epoch 270, Loss 13.710277557373047
Epoch 280, Loss 13.349786758422852
Epoch 

tensor([  5.3671, -17.3012])

* El _loss_ disminuyo mientras fuimos actualizando los parametros en la direccion de _gradient descent_
* No llego a cero
    * Porque?
    
* Sin embargo, los valores de $w$ y $b$ quedaron bastante cerca a los valores necesarios para convertir a Celsius a Fahrenheit.
* Los valores exactos son:
    * $w=5.5556$
    * $b=-17.7778$

In [22]:
import matplotlib.pyplot as plt
%matplotlib inline

t_p = model(t_un, *params)

fig = plt.figure(dpi=600)
plt.xlabel("Fahrenheit")
plt.ylabel("Celsius")
plt.plot(t_u.numpy(), t_p.detach().numpy())
plt.plot(t_u.numpy(), t_c.numpy(), 'o')

ModuleNotFoundError: No module named 'matplotlib'

# PyTorch Autograd

* Back-propagation
    * calcular la gradiente de una composicion de funciones - el modelo y el _loss_ - con respecto a sus parametros - $w$ y $b$ - propagando derivadas hacia atras usando la regla de la cadena.
* Todas las funciones que deben ser diferenciables.
    * Calcular la gradiente: la razon de cambio del _loss_ con respecto a los parametros

**Que pasa cuando tenemos un modelo con millones de parametros?**
* Funcion diferenciable
* calcular la gradiente
* composicion de varias funciones lineales y no lineales

In [25]:
def model(t_u, w, b):
    return w * t_u + b


def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [26]:
params = torch.tensor([1.0, 0.0], requires_grad=True)

`requires_grad=True`: Le estamos diciendo a PyTorch que mantenga un registro de todos los tensores resultantes de operaciones sobre `params`.

Cualquier tensor que tenga `params` como ancestro va a tener acceso a la cadena de funciones que se llamaron para llegar de `params` a ese tensor.

En caso que estas funciones sean diferenciables (la mayoria de operaciones en PyTorch lo son), el valor de la derivada va a ser automaticamente llenado como el atributo `grad` del tensor `params`.

In [27]:
params.grad is None

True

In [28]:
loss = loss_fn(model(t_u, *params), t_c)
loss.backward()

params.grad

tensor([4517.2969,   82.6000])

**dibujo**

* Podemos tener $n$ tensores con `requires_grad=True`
* y cualquier composicion de funciones
* PyTorch calcularia las derivadas del _loss_ a traves de la cadena de esas funciones (grafica computacional)
* **Acumularia** los valores en el atributo `grad` de esos tensores

## Warning

Llamar `backward` hace que las derivadas se **acumulen**. Tenemos que regresar la gradiente a cero explicitamente despues de usarla para actualizar parametros.

In [29]:
if params.grad is not None:
    params.grad.zero_()

In [30]:
def training_loop(model, n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None:
            params.grad.zero_()
            
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        # grad = grad_fn(t_u, t_c, t_p, w, b)
        loss.backward()
        
        params = (params - learning_rate * params.grad).detach().requires_grad_()
        
        if epoch % 200 == 0:
            print(f"Epoch {epoch}, Loss {loss}")
            
    return params

`params = (params - learning_rate * params.grad).detach().requires_grad_()`

Pensemos en la grafica computacional:
* `p1 = (p0 * lr * p0.grad)`
    * `p0` son los _weights_ del modelo
    * `p0.grad` se calcula de una combinacion de `p0` y nuestra data a traves del _loss function_
* `p2 = (p1 * lr * p1.grad)`
* La grafica computacional de `p1` regresa a `p0` esto representa un problema:
    * Tenemos que mantener `p0` en memoria hasta que termine el training
    * Confunde donde tenemos que asignar el error a traves de back-prop
* En vez, despegamos el nuevo tensor de `params` de la grafica computacional usando `detach()`
* De esta forma `params` pierde la memoria de las operaciones que lo generaron.
* Reanudamos el tracking llamando `requires_grad_()`
* Ahora podemos deshacernos de la memoria mantenida por las versiones viejas de `params` y solo hacemos back-prop con los _weights_ actuales.

In [31]:
training_loop(model=model,
              n_epochs=5000,
              learning_rate=1e-2,
              params=torch.tensor([1.0, 0.0], requires_grad=True), # CLAVE
              t_u = t_un, # Seguimos usando la version normalizada
              t_c = t_c)

Epoch 200, Loss 16.608064651489258
Epoch 400, Loss 9.857802391052246
Epoch 600, Loss 6.438284397125244
Epoch 800, Loss 4.706046104431152
Epoch 1000, Loss 3.828537940979004
Epoch 1200, Loss 3.3840172290802
Epoch 1400, Loss 3.1588308811187744
Epoch 1600, Loss 3.0447585582733154
Epoch 1800, Loss 2.986973524093628
Epoch 2000, Loss 2.9576973915100098
Epoch 2200, Loss 2.9428696632385254
Epoch 2400, Loss 2.935356855392456
Epoch 2600, Loss 2.931553840637207
Epoch 2800, Loss 2.929626226425171
Epoch 3000, Loss 2.9286482334136963
Epoch 3200, Loss 2.9281539916992188
Epoch 3400, Loss 2.927903890609741
Epoch 3600, Loss 2.9277760982513428
Epoch 3800, Loss 2.9277124404907227
Epoch 4000, Loss 2.9276793003082275
Epoch 4200, Loss 2.9276628494262695
Epoch 4400, Loss 2.9276552200317383
Epoch 4600, Loss 2.9276490211486816
Epoch 4800, Loss 2.927647590637207
Epoch 5000, Loss 2.9276468753814697


tensor([  5.3671, -17.3012], requires_grad=True)

Ya no tenemos que calcular derivadas a mano :)

## Optimizadores

* Hemos estado utilizando gradient descent normal para optimizacion.
* PyTorch abstrae la estrategia de optimizacion
* El modulo `torch` tiene un submodulo `optim`

In [32]:
import torch.optim as optim

dir(optim)

['ASGD',
 'Adadelta',
 'Adagrad',
 'Adam',
 'Adamax',
 'LBFGS',
 'Optimizer',
 'RMSprop',
 'Rprop',
 'SGD',
 'SparseAdam',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'lr_scheduler']

Todo optmizador:
* toma una lista de parametros (tensores de PyTorch, usualmente con `requires_grad=True`) y mantiene una referencia a ellos.
* luego de que el _loss_ sea calculado con los inputs
* una llamada a `.backward()` provoca que se llene `.grad` en los parametros
* en ese punto, el optimizador puede accesar `.grad` para actualizar los parametros

In [33]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-5
optimizer = optim.SGD([params], lr=learning_rate)

* SGD = Stochastic Gradient Descent
* Exactamente lo mimsmo que hemos estado haciendo a mano (siempre y cuando el argumento `momentum` este en su valor default de 0.0)
* El termino _stochastic_ viene del hecho que la gradiente normalmente se obitene promediando sobre un subset aleatorio de todos los inputs, llamado _minibatch_.

In [34]:
t_p = model(t_u, *params)
loss = loss_fn(t_p, t_c)
loss.backward()

optimizer.step()

params

tensor([ 9.5483e-01, -8.2600e-04], requires_grad=True)

* El valor de `params` se actualizo cuando llamamos `step`
* El optimizador utilizo los valores en `params.grad` y actualizo los parametros restando `(lr * grad)` de ellos.

**Que se nos olvido hacer arriba?**

In [35]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

t_p = model(t_un, *params)
loss = loss_fn(t_p, t_c)

optimizer.zero_grad()
loss.backward()
optimizer.step()

params

tensor([1.7761, 0.1064], requires_grad=True)

In [36]:
def training_loop(model, n_epochs, optimizer, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % 200 == 0:
            print(f"Epoch {epoch}, Loss {loss}")
            
    return params

In [37]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(model,
              n_epochs=5000,
              optimizer=optimizer,
             params = params, # Es importante que ambos `params` sean el mismo objeto
             t_u = t_un,
             t_c = t_c)

Epoch 200, Loss 16.608064651489258
Epoch 400, Loss 9.857802391052246
Epoch 600, Loss 6.438284397125244
Epoch 800, Loss 4.706046104431152
Epoch 1000, Loss 3.828537940979004
Epoch 1200, Loss 3.3840172290802
Epoch 1400, Loss 3.1588308811187744
Epoch 1600, Loss 3.0447585582733154
Epoch 1800, Loss 2.986973524093628
Epoch 2000, Loss 2.9576973915100098
Epoch 2200, Loss 2.9428696632385254
Epoch 2400, Loss 2.935356855392456
Epoch 2600, Loss 2.931553840637207
Epoch 2800, Loss 2.929626226425171
Epoch 3000, Loss 2.9286482334136963
Epoch 3200, Loss 2.9281539916992188
Epoch 3400, Loss 2.927903890609741
Epoch 3600, Loss 2.9277760982513428
Epoch 3800, Loss 2.9277124404907227
Epoch 4000, Loss 2.9276793003082275
Epoch 4200, Loss 2.9276628494262695
Epoch 4400, Loss 2.9276552200317383
Epoch 4600, Loss 2.9276490211486816
Epoch 4800, Loss 2.927647590637207
Epoch 5000, Loss 2.9276468753814697


tensor([  5.3671, -17.3012], requires_grad=True)

## Adam

In [38]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-1
optimizer = optim.Adam([params], lr=learning_rate) # Nuevo optimizador

training_loop(model,
              n_epochs=2000,
              optimizer=optimizer,
              params = params,
              t_u = t_u, # Regresamos a usar el t_u original como input
              t_c = t_c)

Epoch 200, Loss 19.08226203918457
Epoch 400, Loss 10.508145332336426
Epoch 600, Loss 5.641381740570068
Epoch 800, Loss 3.6775739192962646
Epoch 1000, Loss 3.0866997241973877
Epoch 1200, Loss 2.9531853199005127
Epoch 1400, Loss 2.9306914806365967
Epoch 1600, Loss 2.927907705307007
Epoch 1800, Loss 2.927661657333374
Epoch 2000, Loss 2.9276459217071533


tensor([  0.5367, -17.3021], requires_grad=True)

## En resumen

* Back-propagation para estimar la gradiente
* autograd
* optimizar los pesos del modelo usando la SGD u otros optimizadores

# Ejercicios

1. Redefinan el model a `w2 * t_u ** 2 + w1 * t_u + b`
    * Que partes del training loop necesitaron cambiar para acomodar el nuevo modelo?
    * Que partes se mantuvieron iguales?
    * El _loss_ resultante es mas alto o bajo despues de entrenamiento?
    * El resultado es mejor o peor?

In [ ]:
training_loop(model,
              n_epochs=2000,
              optimizer=optimizer,
              params = params,
              t_u = t_u, # Regresamos a usar el t_u original como input
              t_c = t_c)